In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using ForwardDiff
using PyPlot

In [ ]:
Q = Diagonal([0.5; 1])
function f(x)
    return 0.5*(x-[1; 0])'*Q*(x-[1; 0])
end
function ∇f(x)
    return Q*(x-[1; 0])
end
function ∇2f(x)
    return Q
end

In [ ]:
A = [-1.0 1.0]
b = 1.0
function c(x)
    return dot(A,x) - b
end
function ∂c(x)
    return A
end

In [ ]:
function plot_landscape()
    Nsamp = 20
    Xsamp = kron(ones(Nsamp),LinRange(-4,4,Nsamp)')
    Ysamp = kron(ones(Nsamp)',LinRange(-4,4,Nsamp))
    Zsamp = zeros(Nsamp,Nsamp)
    for j = 1:Nsamp
        for k = 1:Nsamp
            Zsamp[j,k] = f([Xsamp[j,k]; Ysamp[j,k]])
        end
    end
    contour(Xsamp,Ysamp,Zsamp)

    xc = LinRange(-4,3,Nsamp)
    plot(xc,xc.+1,"y")
end

plot_landscape()

In [ ]:
function ip_residual(z, ρ)
    x = z[1:2]
    σ = z[3]
    r = [∇f(x) - ∂c(x)'*sqrt(ρ)*exp(-σ);
         c(x) - sqrt(ρ)exp(σ)]
end

In [ ]:
function kkt_residual(z)
    x = z[1:2]
    σ = z[3]
    λ = sqrt(ρ)*exp(-σ)

    r = [∇f(x) - ∂c(x)'*λ;
         min(λ, 0)
         min(c(x),0)
         λ*c(x)]
end

In [ ]:
xguess = [-2; 2]
σguess = 0.0
z = [xguess; σguess]
plot_landscape()
plot(z[1], z[2], "rx")

In [ ]:
ρ = 1.0
ip_residual(z,ρ)

In [ ]:
kkt_residual(z)

In [ ]:
function newton_solve(z0,ρ,tol)

    #initial guess
    z = z0
    
    #KKT residual
    r = ip_residual(z,ρ)

    while norm(r) > tol       
        #H = ∇2f(x)
        #C = ∂c(x)

        #M = [H sqrt(ρ)*C'*exp(-σ);
        #    C -sqrt(ρ)*exp(σ)]

        #Newton step
        M = ForwardDiff.jacobian(dz->ip_residual(dz,ρ), z)
        Δz = -M\r

        znew = z + Δz
        rnew = ip_residual(znew,ρ)

        #Line search
        b = 0.1
        c = 0.5
        α = 1.0
        while norm(rnew) > (norm(r) + b*α*dot(r,M*Δz)/norm(r))
            α = c*α
            znew = z + α*Δz
            rnew = ip_residual(znew,ρ)
        end

        z = znew
        r = rnew
    end

    return z
end

In [ ]:
z_iter = z

In [ ]:
ρ = 1.0e-8 #adjust from ρ=1 to ρ=1e-8 to observe convergence along central path
z = newton_solve(z_iter[:,end],ρ,1e-10)
z_iter = [z_iter z]

In [ ]:
kkt_residual(z)

In [ ]:
plot_landscape()
plot(z_iter[1,:], z_iter[2,:], "rx")

In [ ]:
M = ForwardDiff.jacobian(dz->ip_residual(dz,ρ), z)

In [ ]:
eigvals(M)